# Load all the pig dicom data and convert them to nifti

run from BC network server

In [ ]:
%load_ext autoreload
%autoreload 2



In [ ]:
# !pip install loguru pydicom scipy scikit-image

# !pip install git+https://github.com/mjirik/io3d --upgrade --force-reinstall
# !pip install git+https://github.com/mjirik/imma --upgrade --force-reinstall

# !pip install git+https://github.com/mjirik/io3d
# !pip install git+https://github.com/mjirik/imma



In [ ]:

from pathlib import Path
import sys
import os

io3d_path = Path(os.getcwd()).parent
# io3d_path = Path(__file__).parent.parent.parent
# sys.path.insert(0, str(Path(__file__).parent.parent.parent.resolve()))
print(io3d_path)
io3d_path.exists()


In [ ]:
import io3d
from loguru import logger
from pprint import pprint, pformat
import pandas as pd
import re
import numpy as np
import json
import tqdm.auto

logger.enable("io3d")
force = True
# force = False
force_rewrite = False

recreate_meta = False
recreate_meta = True
# base_path = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_dicom")
dirname = "Prasata DC"
dirname = "Prasata H"
dirname = "Prasata DEC"
dirname = "Prasata I" # partialy done
dirname = "Prasata VC"
dirname = "Prasata VS"
dirname = "Prasata DV" # TODO
dirname = "Prasata ETH" # TODO
dirname = "Prasata HTK" # TODO
dirname = "Prasata K" # TODO
# dirname = "Prasata MASA" # TODO
# dirname = "Prasata Nacvik" # TODO
# dirname = "Prasata Pig N(SSES)" # TODO
# dirname = "Prasata R" # TODO
# dirname = "Prasata ROBO" # TODO
base_path = Path(r"~/mnt/nas-bmc3_ct/" + dirname).expanduser()
fn_prefix = dirname.replace(" ", '_') + "-"
output_dir_part = dirname.replace(" ", '_')
# base_path = Path(r"~/Downloads/_temp/").expanduser()  #: used when the zip files are downloaded manually
# raw_path = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_nii_transposed")
# transposed_path = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_nii_transposed")
# output_path = Path(r"H:\biomedical\orig\pilsen_pigs")
output_path= Path(r"~/mnt/nas-bmc3_ct/pilsen_pigs_all/transplantation_nii_transposed/preprocessed").expanduser() / output_dir_part
common_spreadsheet = output_path / "meta_common.csv"

assert base_path.exists()

# Convert all to nii

In [ ]:

def touch_file(pth:Path):
    try:
        with open(pth, "rb") as f:
            # read just some part of the file
            f.read(1)
    except Exception as e:
        import traceback
        print(f"Error in touching file {pth}: {e}")
        traceback.print_exc()

In [ ]:
from typing import Union

def get_projection(
        datap:io3d.image.DataPlus, axis:Union[int,str], method:str="max"
):
    """Get projection of 3D data to 2D."""
    if isinstance(axis, str):
        dict_axis = {"axial": 0, "coronal": 1, "sagittal": 2}
        if axis in dict_axis:
            axis = dict_axis[axis]
        else:
            raise ValueError(f"Unknown axis {axis}, use one of {list(dict_axis.keys())} or 0, 1, 2")

    data3d = datap.data3d
    axcodes = datap.orientation_axcodes
    data3d = io3d.image.transform_orientation(data3d, axcodes, "SPL")
    if method == "max":
        data2d = data3d.max(axis=axis)
    elif method == "mean":
        data2d = data3d.mean(axis=axis)
    else:
        raise ValueError(f"Unknown method {method}")
    return data2d



In [ ]:
# data2d = get_projection(datap, 0, "max")
# from matplotlib import pyplot as plt
# plt.style.use('classic')
# plt.imshow(data2d, cmap="gray_r")
# plt.colorbar()
# plt.style.available


In [ ]:
# List all directories under base_path (recursively)
# leaf_dirs = []
# for d in tqdm.auto.tqdm(list(base_path.glob("**/"))):
#     if d.is_dir():
#         # Get direct children
#         children = list(d.iterdir())
#         # Identify if there are any subdirectories
#         has_subdir = any(child.is_dir() for child in children)
#         # Identify if there is at least one file in the directory
#         has_file = any(child.is_file() for child in children)
#         # A leaf directory has no subdirectories but does contain files
#         if not has_subdir and has_file:
#             leaf_dirs.append(d)
#
# leaf_dirs

from io3d import fs_tools as fst
leaf_dirs = fst.get_leaf_dirs_with_files(base_path)
leaf_dirs[:5]


In [ ]:
str(leaf_dirs[0].relative_to(base_path))

In [ ]:
# io3d.read(sorted(leaf_dirs)[0], orientation_axcodes="IPL")

In [ ]:
# def get_series_description(dcm_info, series_number):
#     series_description = ""
#     if "SeriesDescription" in dcm_info:
#         series_description = " " + dcm_info["SeriesDescription"]
#
#     return str(series_number) + series_description


In [ ]:
dir_path = sorted(leaf_dirs)[0]

In [ ]:

def get_series_description(dcm_info, series_number):
    series_description = ""
    if "SeriesDescription" in dcm_info:
        series_description = " " + dcm_info["SeriesDescription"]

    return str(series_number) + series_description

def get_stats_of_series_in_dir(dir_path:str):
    dicomdirectory = io3d.dcmreaddata.DicomDirectory(
        str(dir_path)
    )
    return dicomdirectory.get_stats_of_series_in_dir()

def get_stats_of_series_in_dir_nice_list(current_fn, debug=False) -> list:
    series_metadata = get_stats_of_series_in_dir(current_fn)
    for kkey in series_metadata:
        if "dcmfilelist" in series_metadata[kkey]:
            # dir_info = series_metadata
            series_metadata[kkey]["dcmfilelist_len"] = len(series_metadata[kkey]["dcmfilelist"])
            series_metadata[kkey]["dcmfilelist"] = None
        if "SeriesNumber" in series_metadata[kkey]:
            series_metadata[kkey]["SeriesNumber"] = int(series_metadata[kkey]["SeriesNumber"])
        if "Count" in series_metadata[kkey]:
            series_metadata[kkey]["Count"] = int(series_metadata[kkey]["Count"])
        if 'PatientName' in series_metadata[kkey]:
            series_metadata[kkey]["PatientName"] = str(series_metadata[kkey]["PatientName"])
        # check if is json serializable
        series_metadata[kkey]["Path"] = str(current_fn)
        if debug:
            for kkk in series_metadata[kkey].keys():
                logger.debug(f"{kkk}:{type(kkk)}   = {series_metadata[kkey][kkk]}: {type(series_metadata[kkey][kkk])}")
            # logger.debug(series_metadata[kkey].keys())
            # logger.debug(series_metadata[kkey].values())
            series_metadata_list = list(series_metadata.values())
            logger.debug(series_metadata_list)
    series_metadata_list = list(series_metadata.values())
    return series_metadata_list


get_stats_of_series_in_dir_nice_list('/home/mjirik/mnt/nas-bmc3_ct/Prasata VC/2020/VC1 13.8.2020/DICOM/20120714/58550002', True)


In [ ]:

metadata_list = []
for current_fn in sorted(leaf_dirs):
    metadata_ith = get_stats_of_series_in_dir_nice_list(current_fn)
    metadata_list.extend(metadata_ith)


In [ ]:

series_metadata = get_stats_of_series_in_dir_nice_list(sorted(leaf_dirs)[5])
# for key in series_metadata:
#     if "dcmfilelist" in series_metadata[key]:
#         series_metadata[key]["dcmfilelist_len"] = len(series_metadata[key]["dcmfilelist"])
#         series_metadata[key]["dcmfilelist"] = None

series_metadata


In [ ]:
# fnlist = list(base_path.glob("*Tx0*D_V*"))
# find a leaf directory in base_path which contains some file



# fnlist = sorted(list(base_path.glob("*Tx0*D_A*")) + list(base_path.glob("*Tx0*D_V*")))[::-1]
fnlist = sorted(leaf_dirs)


print(f"Number of leaf dirs: {len(fnlist)}")
from joblib import Parallel, delayed
import tqdm
#
stats_subset = []
for fn in tqdm.auto.tqdm(fnlist[:]):
    # iterate over series

    # dir_info = io3d.dcmreaddata.dicomdir_info(fn, gui=False)
    tqdm.tqdm.write(f"Dir {fn}:")
    dir_info = get_stats_of_series_in_dir(fn)
    # for series_number in dir_info.keys():
    #     if 'Count' in dir_info[series_number]:
    #         tqdm.tqdm.write(f"  Series {series_number} with {dir_info[series_number]['Count']} files.")
    #     else:
    #         tqdm.tqdm.write(f'{dir_info[series_number].keys()=}')

    for series_number in dir_info.keys():
        if series_number == None:
            series_number = "first"
        # series_description = get_series_description(dir_info, series_number).replace(" ", "_")

        info_str = ""
        try:
            if "voxelsize_mm" in dir_info[series_number]:
                vs_0 = dir_info[series_number]["voxelsize_mm"][0]
                # print(f"{dir_info[series_number]["voxelsize_mm"]=}")
                info_str += f"{vs_0:.2f}"
        except Exception as e:
            tqdm.tqdm.write(f"voxelsize problem")
            tqdm.tqdm.write(traceback.format_exc())
        try:
            if 'StudyDescription' in dir_info[series_number]:
                study_description = str(dir_info[series_number]["StudyDescription"])
                study_description = study_description.replace(" ", "_").replace("(", "_").replace(")", "_")
                study_description_lower = study_description.lower()
                info_str += f"_{study_description_lower}"[:35]
        except Exception as e:
            tqdm.tqdm.write(f"study description problem")
            tqdm.tqdm.write(traceback.format_exc())

        # i want to have there the parent directory
        # fn_name = str(fn.relative_to(base_path)).replace("\\", "-").replace("/", "-")
        fn_name = (fn_prefix + str(fn.relative_to(base_path))).replace("\\", "-").replace("/", "-").replace(" ", "_")
        fn_name += f"_{series_number}"
        if info_str != "":
            fn_name += "_"
            fn_name += info_str

        # logger.info(fn)
        fn_in = fn
        # fn_out = raw_path / fn.name / f"{fn.name}.mhd"
        fn_out = output_path / fn_name / f"{fn_name}.nii.gz"
        if fn_out.exists() and not force_rewrite:
            tqdm.tqdm.write(f"File {fn_out.name} already exists. skipping.")
            continue

        fn_meta = fn_out.parent / "meta.json"
        if force or (not fn_out.exists()):
            # if count of files is lower than 2, skip
            if dir_info[series_number]["Count"] <2:
                continue


            try:
                # tqdm.tqdm.write(f"Reading series {series_number} with {dir_info[series_number]["Count"]} files in {fn_in} and writing {fn_out}")
                # tqdm.tqdm.write(f"Number of files in the directory: {len(fn_fns)}")
                # Parallel(n_jobs=2)(delayed(touch_file)(fn) for fn in tqdm.tqdm(fn_fns, desc="touching files"))

                axcodes = "IPL"
                # logger.debug(f"Reading {fn_in} with axcodes={axcodes}")
                datap = io3d.read(fn_in,
                                  # series_number="first",
                                  series_number=series_number,
                                  orientation_axcodes=axcodes)
                # logger.debug(datap.keys())
                fn_out.parent.mkdir(parents=True, exist_ok=True)
                io3d.write(datap, fn_out)
                # logger.debug("writing done, creating projections")
                for axis in ["axial", "coronal", "sagittal"]:
                        data2d = get_projection(datap, axis, "max")
                        import skimage.io
                        # change intensity to range 0..1
                        data2d = data2d.astype(float)
                        data2d_min = np.min(data2d)
                        data2d_max = np.max(data2d)
                        data2d = (255 * (data2d - data2d_min) / (data2d_max - data2d_min))
                        data2d = data2d.astype(np.uint8)
                        skimage.io.imsave(fn_out.parent / f"{fn_out.stem}_{axis}.jpg", data2d)

                stats_i = get_stats_of_series_in_dir_nice_list(fn_in)
                stats_subset.extend(stats_i)
                try:
                    with open(fn_out.parent / f"{fn_out.stem}_stats.json", "w") as f:
                        json.dump(stats_i, f)
                except TypeError as e:
                    tqdm.tqdm.write(f"Error in JSON export {fn_in}: {e}")
                    tqdm.tqdm.write(traceback.format_exc())
                    tqdm.tqdm.write(str(stats_i))
                    get_stats_of_series_in_dir_nice_list(fn_in, debug=True)

                # logger.debug("projections done")
            except Exception as e:
                import traceback
                tqdm.tqdm.write(f"Error in reading {fn_in}: {e}")
                tqdm.tqdm.write(traceback.format_exc())
            if len(list(fn_out.parent.glob("*"))) == 0:
                fn_out.parent.rmdir()
                # logger.debug(f"shape={datap.data3d.shape}, {datap.orientation_axcodes}")
            # with open(fn_meta, "w") as f:
            #     json.dump(dict(row), f)


df_subset = pd.DataFrame(stats_subset)
df_subset.to_csv(base_path / "meta.csv", index=False)




In [ ]:
# import tqdm
# import time
# for i in tqdm.tqdm(range(100)):
#     #wait 0.1s
#     time.sleep(0.1)


In [ ]:
if common_spreadsheet.exists():
    df = pd.read_csv(common_spreadsheet)
    # merge records df_common + df_subset, keep columns
    df = pd.concat([df, df_subset], ignore_index=True, verify_integrity=True)
else:
    df = pd.DataFrame(stats_subset)

df.to_csv(common_spreadsheet, index=False)




In [ ]:
# def create_meta(base_path, metafn):
#     data = {
#         "dirname": [],
#         "dataset_type": [],
#         "id": [],
#     }
#     i_train = 0
#     i_test = 0
#     i_val = 0
#     fnlist = base_path.glob("*Tx0*D_V*")
#     logger.info(pformat(list(map(str, list(fnlist)))))
#     fnlist = base_path.glob("*Tx0*D_V*")
#     for fn in fnlist:
#         ia = int(re.findall(r"([0-9]+)", fn.name)[0])
#         ii = None
#         tp = None
#         if (ia % 4) == 0:
#             i_test += 1
#             ii = i_test
#             tp = "test"
#         elif (ia % 2) == 0:
#             i_val += 1
#             ii = i_val
#             tp = "val"
#         else:
#             i_train += 1
#             ii = i_train
#             tp = "train"
#
#         data["dirname"].append(fn.name)
#         data["dataset_type"].append(tp)
#         data["id"].append(ii)
#
#     df = pd.DataFrame(data)
#     df.to_csv(metafn, index=None)



In [ ]:
# if recreate_meta:
#     create_meta(base_path, metafn)


In [ ]:
# fnlist = base_path.glob("*debug*")
# fnlist = base_path.glob("*Tx041D_V*")


# for i, row in tqdm.tqdm(df.iterrows()):
#     fn_in = base_path / row["dirname"]
#     fn_out = (
#             output_path
#             / row["dataset_type"]
#             / f"PP_{row['id']:04}"
#             / "PATIENT_DICOM"
#             / f"PP_{row['id']:04}.mhd"
#     )
#     fn_meta = fn_out.parent / "meta.json"
#     logger.debug(fn_in)
#     logger.debug(fn_out)
#
#     fn_out.parent.mkdir(parents=True, exist_ok=True)
#     if (
#             not fn_meta.exists()
#     ):  # we do not need to read the data again if everything is done. We are checkin:w
#
#         tqdm.tqdm.write(f"Reading {fn_in} and writing {fn_out}")
#         datap = io3d.read(fn_in)
#         io3d.write(datap, fn_out)
#         with open(fn_meta, "w") as f:
#             json.dump(dict(row), f)
